# Setup

In [1]:
import os
import numpy as np
import pandas as pd
import swifter
import re
import requests

from tqdm.notebook import tqdm
tqdm.pandas()

from bs4 import BeautifulSoup

In [2]:
HTML_PATH = 'T:/New Download Folder/anime_html'

In [4]:
df = pd.read_csv('data/anime_with_synopsis.csv')
df.head()

,MAL_ID,Name,Score,Genres,synopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


In [5]:
df.synopsis[1]

'other day, another bounty—such is the life of the often unlucky crew of the Bebop. However, this routine is interrupted when Faye, who is chasing a fairly worthless target on Mars, witnesses an oil tanker suddenly explode, causing mass hysteria. As casualties mount due to a strange disease spreading through the smoke from the blast, a whopping three hundred million woolong price is placed on the head of the supposed perpetrator. With lives at stake and a solution to their money problems in sight, the Bebop crew springs into action. Spike, Jet, Faye, and Edward, followed closely by Ein, split up to pursue different leads across Alba City. Through their individual investigations, they discover a cover-up scheme involving a pharmaceutical company, revealing a plot that reaches much further than the ragtag team of bounty hunters could have realized.'

# Scrape full synopsis

In [28]:
mal_id = 34572
html_path = f'{HTML_PATH}/{mal_id}/details.html'
with open(html_path, 'r', encoding='utf-8') as f:
    html = f.read()
soup = BeautifulSoup(html)
# r = requests.get(f'https://myanimelist.net/anime/{mal_id}')
# soup = BeautifulSoup(r.content)
# img_url = soup.find('img', attrs={'class': 'ac'})['data-src']
synopsis = soup.find('p', attrs={'itemprop': 'description'}).get_text()
synopsis

'\n              Asta and Yuno were abandoned at the same church on the same day. Raised together as children, they came to know of the "Wizard King"—a title given to the strongest mage in the kingdom—and promised that they would compete against each other for the position of the next Wizard King. However, as they grew up, the stark difference between them became evident. While Yuno is able to wield magic with amazing power and control, Asta cannot use magic at all and desperately tries to awaken his powers by training physically.\n              \n\n              When they reach the age of 15, Yuno is bestowed a spectacular Grimoire with a four-leaf clover, while Asta receives nothing. However, soon after, Yuno is attacked by a person named Lebuty, whose main purpose is to obtain Yuno\'s Grimoire. Asta tries to fight Lebuty, but he is outmatched. Though without hope and on the brink of defeat, he finds the strength to continue when he hears Yuno\'s voice. Unleashing his inner emotions 

In [3]:
def get_synopsis(mal_id):
    # mal_id = get_anime_rows(synopsis_df, anime_name).iloc[0, 0]
    # r = requests.get(f'https://myanimelist.net/anime/{mal_id}')
    # same with requesting from URL but faster
    html_path = f'{HTML_PATH}/{mal_id}/details.html'
    with open(html_path, 'r', encoding='utf-8') as f:
        html = f.read()
    soup = BeautifulSoup(html)
    synopsis = soup.find('p', attrs={'itemprop': 'description'}).get_text()
    return synopsis

In [4]:
def get_img_url(mal_id):
    html_path = f'{HTML_PATH}/{mal_id}/details.html'
    with open(html_path, 'r', encoding='utf-8') as f:
        html = f.read()
    soup = BeautifulSoup(html)
    try:
        img_url = soup.find('img', attrs={'class': 'ac'})['data-src']
    except:
        print(f"Image URL not found for Anime ID: {mal_id}")
        return np.nan
    return img_url

In [5]:
def get_img_synopsis(mal_id):
    html_path = os.path.join(HTML_PATH, str(mal_id), 'details.html')
    with open(html_path, 'r', encoding='utf-8') as f:
        html = f.read()
    soup = BeautifulSoup(html)
    synopsis = soup.find('p', attrs={'itemprop': 'description'}).get_text()
    img_url = soup.find('img', attrs={'class': 'ac'})['data-src']
    return img_url, synopsis

In [13]:
%%time
df['new_synopsis'] = df.MAL_ID.apply(get_synopsis)

Wall time: 39min 28s


In [63]:
df.head()

,MAL_ID,Name,Score,Genres,synopsis,new_synopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever...","\n In the year 2071, humanity has..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ...","\n Another day, another bounty—su..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0...",\n Vash the Stampede is the man w...
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...,\n Witches are individuals with s...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...,\n It is the dark century and the...


In [71]:
df.loc[4344]

MAL_ID                                                       7495
Name                               Nonki na Tou-san Ryuuguu Mairi
Score                                                     Unknown
Genres                                               Supernatural
synopsis        One of the first examples of the tale is Hakuz...
new_synopsis    \n              One of the first examples of t...
Name: 4344, dtype: object

In [73]:
%%time
df['img_url'] = df.MAL_ID.progress_apply(get_img_url)

  0%|          | 0/16214 [00:00<?, ?it/s]

Image URL not found for Anime ID: 7495
Image URL not found for Anime ID: 7500
Image URL not found for Anime ID: 7527
Image URL not found for Anime ID: 10756
Image URL not found for Anime ID: 10758
Image URL not found for Anime ID: 10759
Image URL not found for Anime ID: 17387
Image URL not found for Anime ID: 21761
Image URL not found for Anime ID: 21763
Image URL not found for Anime ID: 22219
Image URL not found for Anime ID: 22391
Image URL not found for Anime ID: 22511
Image URL not found for Anime ID: 22995
Image URL not found for Anime ID: 23097
Image URL not found for Anime ID: 23181
Image URL not found for Anime ID: 23183
Image URL not found for Anime ID: 23185
Image URL not found for Anime ID: 23187
Image URL not found for Anime ID: 23191
Image URL not found for Anime ID: 23635
Image URL not found for Anime ID: 23637
Image URL not found for Anime ID: 23641
Image URL not found for Anime ID: 23651
Image URL not found for Anime ID: 23911
Image URL not found for Anime ID: 23971
Ima

In [74]:
df.img_url.isna().sum()

146

In [62]:
df.new_synopsis.isna().sum()

0

In [75]:
df

,MAL_ID,Name,Score,Genres,synopsis,new_synopsis,img_url
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever...","\n In the year 2071, humanity has...",https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ...","\n Another day, another bounty—su...",https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0...",\n Vash the Stampede is the man w...,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...,\n Witches are individuals with s...,https://cdn.myanimelist.net/images/anime/1796/...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...,\n It is the dark century and the...,https://cdn.myanimelist.net/images/anime/7/215...
...,...,...,...,...,...,...,...
16209,48481,Daomu Biji Zhi Qinling Shen Shu,Unknown,"Adventure, Mystery, Supernatural",No synopsis information has been added to this...,\n No synopsis information has be...,https://cdn.myanimelist.net/images/anime/1166/...
16210,48483,Mieruko-chan,Unknown,"Comedy, Horror, Supernatural",ko is a typical high school student whose life...,\n Miko is a typical high school ...,https://cdn.myanimelist.net/images/anime/1385/...
16211,48488,Higurashi no Naku Koro ni Sotsu,Unknown,"Mystery, Dementia, Horror, Psychological, Supe...",Sequel to Higurashi no Naku Koro ni Gou .,\n Sequel to\n \n ...,https://cdn.myanimelist.net/images/anime/1721/...
16212,48491,Yama no Susume: Next Summit,Unknown,"Adventure, Slice of Life, Comedy",New Yama no Susume anime.,\n New\n \n ...,https://cdn.myanimelist.net/images/anime/1998/...


In [77]:
df.img_url.head().values

array(['https://cdn.myanimelist.net/images/anime/4/19644.jpg',
       'https://cdn.myanimelist.net/images/anime/1439/93480.jpg',
       'https://cdn.myanimelist.net/images/anime/7/20310.jpg',
       'https://cdn.myanimelist.net/images/anime/1796/91065.jpg',
       'https://cdn.myanimelist.net/images/anime/7/21569.jpg'],
      dtype=object)

In [170]:
df.new_synopsis = df.new_synopsis.str.strip()

In [179]:
df.new_synopsis.replace(r'(\s{2,}|\n)', ' ', regex=True, inplace=True)

In [13]:
df['new_synopsis'].sample(3).values

array(["Music video for millennium parade's song Philip .",
       "After coming across an advertisement about how anyone can lead an invasion force, young Don decides to establish a ragtag squad of his own to conquer the cosmos. Working alongside him are the aloof Pepu and the gaming-obsessed Chiku, though the value of their membership remains somewhat questionable. With his two henchmen by his side, the crew's inexperienced leader sets his sights on their first target: Earth. According to the manuals, Earth should be a simple enough target, but the situation spirals out of control right from the start. As it turns out, convincing over seven billion humans to simply roll over and accept their new overlord is no easy feat. Having been blown off by the entire human species, Don and his squadron touch down on Earth and begin their search for a way to subjugate humanity. However, with their unexpected failure, they face more pressing issues—namely, finding food and paying rent—so their pl

In [183]:
# df.to_csv('data/anime_info.csv', index=False)

# Combine for more Anime info

In [3]:
df = pd.read_csv('data/anime_info.csv')
df.head()

,MAL_ID,Name,Score,Genres,new_synopsis,img_url
0,5114,Fullmetal Alchemist: Brotherhood,9.19,"Action, Military, Adventure, Comedy, Drama, Ma...","""In order for something to be obtained, someth...",https://cdn.myanimelist.net/images/anime/1223/...
1,40028,Shingeki no Kyojin: The Final Season,9.17,"Action, Military, Mystery, Super Power, Drama,...",Gabi Braun and Falco Grice have been training ...,https://cdn.myanimelist.net/images/anime/1000/...
2,9253,Steins;Gate,9.11,"Thriller, Sci-Fi",The self-proclaimed mad scientist Rintarou Oka...,https://cdn.myanimelist.net/images/anime/5/731...
3,38524,Shingeki no Kyojin Season 3 Part 2,9.10,"Action, Drama, Fantasy, Military, Mystery, Sho...",Seeking to restore humanity's diminishing hope...,https://cdn.myanimelist.net/images/anime/1517/...
4,28977,Gintama°,9.10,"Action, Comedy, Historical, Parody, Samurai, S...","Gintoki, Shinpachi, and Kagura return as the f...",https://cdn.myanimelist.net/images/anime/3/720...


In [28]:
len(df)

16214

In [7]:
df.loc[df.MAL_ID == 34572, 'new_synopsis'].values

array(['Asta and Yuno were abandoned at the same church on the same day. Raised together as children, they came to know of the "Wizard King"—a title given to the strongest mage in the kingdom—and promised that they would compete against each other for the position of the next Wizard King. However, as they grew up, the stark difference between them became evident. While Yuno is able to wield magic with amazing power and control, Asta cannot use magic at all and desperately tries to awaken his powers by training physically. When they reach the age of 15, Yuno is bestowed a spectacular Grimoire with a four-leaf clover, while Asta receives nothing. However, soon after, Yuno is attacked by a person named Lebuty, whose main purpose is to obtain Yuno\'s Grimoire. Asta tries to fight Lebuty, but he is outmatched. Though without hope and on the brink of defeat, he finds the strength to continue when he hears Yuno\'s voice. Unleashing his inner emotions in a rage, Asta receives a five-leaf clove

In [65]:
anime_df = pd.read_csv('data/anime.csv', dtype={'Score': str})
anime_df

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26.0,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1.0,"Sep 1, 2001",NaN,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26.0,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26.0,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17557,48481,Daomu Biji Zhi Qinling Shen Shu,NaN,"Adventure, Mystery, Supernatural",NaN,盗墓笔记之秦岭神树,ONA,NaN,"Apr 4, 2021 to ?",NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
17558,48483,Mieruko-chan,NaN,"Comedy, Horror, Supernatural",NaN,見える子ちゃん,TV,NaN,2021 to ?,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17559,48488,Higurashi no Naku Koro ni Sotsu,NaN,"Mystery, Dementia, Horror, Psychological, Supe...",Higurashi:When They Cry – SOTSU,ひぐらしのなく頃に卒,TV,NaN,"Jul, 2021 to ?",Summer 2021,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17560,48491,Yama no Susume: Next Summit,NaN,"Adventure, Slice of Life, Comedy",NaN,ヤマノススメ Next Summit,TV,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
anime_df.Score

0        8.78
1        8.39
2        8.24
3        7.27
4        6.98
         ... 
17557     NaN
17558     NaN
17559     NaN
17560     NaN
17561     NaN
Name: Score, Length: 17562, dtype: float64

In [9]:
anime_df.columns

Index(['MAL_ID', 'Name', 'Score', 'Genres', 'English name', 'Japanese name',
       'Type', 'Episodes', 'Aired', 'Premiered', 'Producers', 'Licensors',
       'Studios', 'Source', 'Duration', 'Rating', 'Ranked', 'Popularity',
       'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
       'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1'],
      dtype='object')

In [7]:
def get_anime_rows(df, anime_name, exact_name=False, types=None):
    df = df.copy()
    if exact_name:
        # get exact name
        df = df[df.Name == anime_name]
    else:
        df = df[df.Name.str.contains(anime_name, case=False, regex=False) |
                df['English name'].str.contains(anime_name, case=False, regex=False)]

    if types:
        try:
            types = set([types]) if isinstance(types, str) else set(types)
            df = df[df.Type.isin(types)]
        except:
            raise Exception('Anime type not valid!')
    return df

In [29]:
combined = pd.merge(df, anime_df, on='MAL_ID', suffixes=('', '_y'))

In [30]:
combined.columns

Index(['MAL_ID', 'Name', 'Score', 'Genres', 'new_synopsis', 'img_url',
       'Name_y', 'Score_y', 'Genres_y', 'English name', 'Japanese name',
       'Type', 'Episodes', 'Aired', 'Premiered', 'Producers', 'Licensors',
       'Studios', 'Source', 'Duration', 'Rating', 'Ranked', 'Popularity',
       'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
       'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1'],
      dtype='object')

In [31]:
combined = combined[['MAL_ID', 'Name', 'English name', 'Score', 'Genres', 'Type', 'Episodes', 'Aired', 'Popularity', 'new_synopsis', 'img_url']]

In [87]:
# combined = pd.read_csv('data/full_anime_info.csv')

In [88]:
combined

,MAL_ID,Name,English name,Score,Genres,Type,Episodes,Aired,Popularity,new_synopsis,img_url
0,20707,"""0""",Unknown,4.71,Music,Music,1,"Oct 23, 2013",7736,This music video tells how a shy girl with a s...,https://cdn.myanimelist.net/images/anime/6/548...
1,25627,"""Aesop"" no Ohanashi yori: Ushi to Kaeru, Yokub...",Unknown,NaN,Kids,Movie,1,"Mar 21, 1970",13082,Based on Aesop's Fables.,https://cdn.myanimelist.net/images/anime/3/651...
2,7669,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",Unknown,6.91,"Comedy, Fantasy, School",OVA,1,"Dec 26, 2009",3876,Short episode bundled with the limited edition...,https://cdn.myanimelist.net/images/anime/2/799...
3,8481,"""Bungaku Shoujo"" Memoire",Unknown,7.34,"Drama, Romance, School",OVA,3,"Jun 25, 2010 to Dec 24, 2010",3259,Episodes which depict the background stories o...,https://cdn.myanimelist.net/images/anime/6/267...
4,6408,"""Bungaku Shoujo"" Movie",Unknown,7.41,"Mystery, Drama, Romance, School",Movie,1,"May 1, 2010",2049,"The protagonist of the story, Konoha Inoue, is...",https://cdn.myanimelist.net/images/anime/8/811...
...,...,...,...,...,...,...,...,...,...,...,...
16209,6864,xxxHOLiC Rou,Unknown,8.23,"Mystery, Supernatural",OVA,2,"Apr 23, 2010 to Mar 9, 2011",2066,10 years after the events of xxxHOLiC Shunmuki...,https://cdn.myanimelist.net/images/anime/9/250...
16210,4918,xxxHOLiC Shunmuki,Unknown,8.08,"Mystery, Comedy, Psychological, Supernatural, ...",OVA,2,"Feb 17, 2009 to Jun 23, 2009",2026,"For the appropriate price, your dearest wish c...",https://cdn.myanimelist.net/images/anime/12/25...
16211,29708,Üks Uks,Unknown,NaN,Dementia,Movie,1,2003,15229,Opening one's inner doors of conflict is the k...,https://cdn.myanimelist.net/images/anime/11/71...
16212,32878,ēlDLIVE,elDLIVE,6.16,"Action, Sci-Fi, Space, Police, Shounen",TV,12,"Jan 8, 2017 to Mar 26, 2017",2483,Chuuta Kokonose is an orphan who lives with hi...,https://cdn.myanimelist.net/images/anime/8/823...


In [69]:
combined.to_csv('data/full_anime_info.csv', index=False)